In [1]:
import os

In [9]:
%pwd

'C:\\Users\\HP\\End-to-End-cnnclassification-using-mlflow-DVC'

os.getcwd()

In [4]:
os.chdir("/..")

In [6]:
%pwd

'c:\\'

In [4]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        ) 

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [11]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        """Fetch data from the url
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
            """
           zip_file_path: str
           Extracts the zip file into the data directory
           Function returns None
        """
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-14 13:44:20,772: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-14 13:44:20,777: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-14 13:44:20,778: INFO: common: created directory at: artifacts]
[2024-02-14 13:44:20,781: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-14 13:44:20,784: INFO: 3758042430: Download data from https://drive.google.com/file/d/1Y3f6Oifu2C3py_MD5bB8FifXLJ6eGKWC/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1Y3f6Oifu2C3py_MD5bB8FifXLJ6eGKWC
To: C:\Users\HP\End-to-End-cnnclassification-using-mlflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 2.29M/2.29M [00:00<00:00, 10.3MB/s]

[2024-02-14 13:44:25,121: INFO: 3758042430: Download data from https://drive.google.com/file/d/1Y3f6Oifu2C3py_MD5bB8FifXLJ6eGKWC/view?usp=sharing into file artifacts/data_ingestion/data.zip]


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'artifacts/data_ingestion'